In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

data = pd.read_csv("./review_pizza.csv",index_col=0)
#delete "date"
data.pop('date')
#sort by business_id.
data=data.sort_values(by='business_id')

stars=pd.np.array(data['stars']).tolist()
business_id=pd.np.array(data['business_id']).tolist()
t=pd.np.array(data['text']).tolist()

for i in range(data.shape[0]):
    t[i]=t[i].lower()

def check_word (word,source):
    res=[]
    for i in range(data.shape[0]):
        res.append(word in source[i])
    return sum(res)
#Then check some words, omit here.

for doc in range(data.shape[0]):
    t[doc]=re.sub('5\sstar[s]*','exellent',t[doc])
    t[doc]=re.sub('five\sstar[s]*','exellent',t[doc])
    t[doc]=re.sub('4\sstar[s]*','good',t[doc])
    t[doc]=re.sub('four\sstar[s]*','good',t[doc])
    t[doc]=re.sub('3\sstar[s]*','average',t[doc])
    t[doc]=re.sub('three\sstar[s]*','average',t[doc])
    t[doc]=re.sub('[12]\sstar[s]*','bad',t[doc])
    t[doc]=re.sub('two\sstar[s]*','bad',t[doc])
    t[doc]=re.sub('one\sstar','bad',t[doc])
    t[doc]=re.sub('[:;]+["^-]*\)+','pleased',t[doc])
    t[doc]=re.sub('[:;]+["^-]*\(+','frustrated',t[doc])
    t[doc]=re.sub('n\'t','not',t[doc])
    t[doc]=re.sub('\'d','',t[doc])
    t[doc]=re.sub('\'m','',t[doc])
    t[doc]=re.sub('\'re','',t[doc])
    t[doc]=re.sub('\'ll','',t[doc])
    t[doc]=re.sub('\'s','',t[doc])
    t[doc]=re.sub('[^A-Za-z0-9\s]','',t[doc])
    t[doc]=re.sub(r'\d+ (?!min)','',t[doc])
    t[doc]=re.sub('\n','',t[doc])
    
def word_split (doc):
    doc=re.split('\s*',doc)
    return doc

word_matrix=[]
for i in range(len(t)):
    word_matrix.append(word_split(t[i]))
#actualy it's a list of lists

wordnet_lemmatizer = WordNetLemmatizer()
for i in range(len(word_matrix)):
    for j in range(len(word_matrix[i])):
        word_matrix[i][j]=wordnet_lemmatizer.lemmatize(word_matrix[i][j])
        
stop_words = set(stopwords.words('english'))
m=[]
for i in range(len(word_matrix)):
    m.append([w for w in word_matrix[i] if not w in stop_words])
    
cleaned_words=[]
for i in range(len(m)):
    cleaned_words.append([word for word in m[i] if len(word)>=3])

length=len(sorted(cleaned_words,key=len, reverse=True)[0])
cleaned_m=np.array([xi+[None]*(length-len(xi)) for xi in cleaned_words])
df=pd.concat([pd.Series(x) for x in cleaned_m], axis=1)
corpus=df.transpose()
corpus.insert(loc=0,column='stars',value=stars)
corpus.insert(loc=0,column='business_id',value=business_id)
corpus.to_csv('sorted_corpus.csv',index=False)